In [1]:
import os
import os.path as op
import json
import cv2
import base64
import numpy as np
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw

from maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer
from maskrcnn_benchmark.structures.tsv_file_ops import generate_linelist_file
from maskrcnn_benchmark.structures.tsv_file_ops import generate_hw_file
from maskrcnn_benchmark.structures.tsv_file import TSVFile
from maskrcnn_benchmark.data.datasets.utils.image_ops import img_from_base64

In [2]:
orig_root = '../../../../datasets/coco_my_ocr_13'

img_root = op.join(orig_root, 'images')
cap_json = op.join(orig_root, 'captions_val2014.json')

In [3]:
img_subdir = img_root #op.join(img_root, split + '2014')
img_list = sorted(os.listdir(img_subdir))

print(len(img_list))

13


In [4]:
with open(cap_json) as f:
    captions = json.load(f)
captions = captions['annotations']

In [5]:
captions[0]

{'image_id': 203564,
 'id': 37,
 'caption': 'A bicycle replica with a clock as the front wheel.'}

In [6]:
folder_img_ids = set(img_name[13:-4].lstrip('0') for img_name in img_list)

In [8]:
folder_img_ids

{'1532',
 '1668',
 '1779',
 '2521',
 '2525',
 '3799',
 '4093',
 '4497',
 '5595',
 '5860',
 '641',
 '7115',
 '730'}

In [11]:
out_captions = []
for cap_sample in captions:
    if str(cap_sample['image_id']) in folder_img_ids:
        out_captions.append(cap_sample)

out_captions = sorted(out_captions, key=lambda x:x['image_id'])

for cap in out_captions:
    cap['image_id'] = str(cap['image_id'])

out_captions[:3]

[{'image_id': '641',
  'id': 691034,
  'caption': 'A bus that is sitting in the street.'},
 {'image_id': '641',
  'id': 694850,
  'caption': 'A long white bus driving down a street.'},
 {'image_id': '641',
  'id': 701663,
  'caption': 'A large passenger bus going down a city street. '}]

In [15]:
# Generate captions in COCO format

idim = []
for cap in out_captions:
    idim.append({'id': cap['image_id'], 'file_name': cap['image_id']})

out_captions_coco_fmt = {'annotations': out_captions, 'images': idim, 'type': 'captions', 'info': 'dummy', 'licenses': 'dummy'}

In [16]:
# Save JSON

with open(os.path.join(orig_root, 'captions.json'), 'w') as fp:
    json.dump(out_captions, fp)
    
with open(os.path.join(orig_root, 'captions_coco_format.json'), 'w') as f:
    json.dump(out_captions_coco_fmt, f)

In [13]:
# Save TSV

rows = []

for sample in out_captions:
    row = [sample['image_id'], sample['caption'] ]
    rows.append(row)

exp_tsv_file = op.join(orig_root, 'captions.tsv')
tsv_writer(rows, exp_tsv_file)